# Discrete Event Simulation with Mesa

## Tutorial Description

This tutorial introduces Mesa's **Discrete Event Simulation (DEVS)** capabilities using the `DEVSimulator`. You'll learn how to model systems where events occur at specific points in time, rather than in fixed time steps.

**Learning Objectives:**
- Understand the difference between tick-based and event-based simulation
- Learn when to use DEVS instead of traditional ABM
- Master the `DEVSimulator` for pure event scheduling
- Apply queuing theory to validate simulation results
- Build an M/M/1 queue system from scratch

**Prerequisites:**
- Complete Tutorial 0 (First Model)
- Complete Tutorial 1 (Adding Space) 
- Complete Tutorial 2 (Collecting Data)
- Basic understanding of probability distributions

**What You'll Build:**
A bank queue system with random customer arrivals and service times, analyzing wait times and queue lengths using both simulation and theoretical predictions.

**Important:** 
- If you are just exploring Mesa and want the fastest way to execute the code we recommend executing this tutorial online in a Colab notebook. [![Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/projectmesa/mesa/blob/main/docs/tutorials/11_discrete_event_simulation.ipynb)
- If you do not have a Google account you can use [![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/projectmesa/mesa/main?labpath=docs%2Ftutorials%2F11_discrete_event_simulation.ipynb)
- If you are running locally, please ensure you have the latest Mesa version installed.

### Tutorial Setup

If running locally, ensure Mesa is installed:

```bash
pip install mesa[rec]
```

For this tutorial, we'll also use:
```bash
pip install numpy pandas seaborn
```

## What is Discrete Event Simulation?

In traditional agent-based models (ABM), time advances in **fixed increments** called ticks. At each tick, all agents are activated:

**Tick-Based ABM:**
- Time: 0, 1, 2, 3, 4, ... (discrete, uniform steps)
- Every agent's `step()` method is called each tick
- Works well when agents are frequently active

**Event-Based Simulation (DEVS):**
- Time: 0, 1.3, 2.7, 5.2, ... (discrete, non-uniform)
- Only agents/events scheduled for specific times execute
- Efficient when agents are dormant for long periods

In [ ]:
import mesa
from mesa.experimental.devs import DEVSimulator


# Example: Traditional tick-based model
class TickBasedModel(mesa.Model):
    def step(self):
        # ALL agents activated EVERY tick - even if doing nothing!
        for agent in self.agents:
            if agent.has_work_to_do():
                agent.do_work()
            # Problem: We check even when agent is idle


# Example: Event-based model
class EventBasedModel(mesa.Model):
    def __init__(self):
        super().__init__()
        self.simulator = DEVSimulator()

        # Only schedule events when agent actually has work
        for agent in self.agents:
            if agent.has_work_to_do():
                # Schedule the exact time this agent needs to act
                self.simulator.schedule_event_absolute(
                    agent.do_work, time=agent.work_time
                )

### When Should You Use DEVS?

**Use DEVS when:**
- Agents spend long periods dormant (e.g., waiting, sleeping, traveling)
- Events occur at irregular intervals (arrivals, failures, completions)
- You need continuous time (non-integer time values)
- Performance matters and most agents are inactive
- Modeling queuing systems, manufacturing, logistics

**Use Traditional ABM when:**
- Most agents are active every time step
- Time steps are naturally discrete (e.g., days, rounds)
- Agent interactions happen synchronously
- Simplicity is more important than efficiency

### Real-World DEVS Applications

1. **Queuing Systems**: Bank tellers, call centers, checkout lines
2. **Manufacturing**: Machine failures, repair schedules, production runs
3. **Healthcare**: Patient arrivals, surgery durations, bed availability
4. **Transportation**: Traffic lights, vehicle arrivals, route planning
5. **Computer Systems**: Packet routing, process scheduling, server requests

In [ ]:
# Import required libraries
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

# Mesa imports
import mesa
from mesa.experimental.devs import DEVSimulator, Priority

# Set random seed for reproducibility
np.random.seed(42)

## Mathematical Foundations: Queuing Theory

Before building our simulation, let's understand the **mathematics** behind queuing systems.

### The M/M/1 Queue

Our bank queue is an **M/M/1 system**:
- **M** (Markov): Arrivals follow a Poisson process (exponential inter-arrival times)
- **M** (Markov): Service times are exponentially distributed
- **1**: One server

### Key Parameters

**Arrival Rate (λ)**: Average number of customers arriving per time unit  
**Service Rate (μ)**: Average number of customers served per time unit  

**Example**: If λ = 2 customers/hour and μ = 3 customers/hour, the server can handle the load.

### System Utilization (ρ)

The **utilization** is the fraction of time the server is busy:

$$\rho = \frac{\lambda}{\mu}$$

**Critical Condition**: For a stable system, we need **ρ < 1**  
- If ρ >= 1, arrivals occur faster than service -> queue grows infinitely!
- If ρ = 0.67, server is busy 67% of the time

**Check**: With λ = 2, μ = 3:
$$\\rho = \frac{2}{3} = 0.67$$ (Stable system)

### Theoretical Performance Metrics

For a stable M/M/1 queue (ρ < 1), we can **predict** performance:

**Average Number in System**:
$$L = \\frac{\\rho}{1 - \\rho} = \\frac{\\lambda}{\\mu - \\lambda}$$

**Average Time in System**:
$$W = \\frac{1}{\\mu - \\lambda}$$

**Average Queue Length**:
$$L_q = \\frac{\\rho^2}{1 - \\rho} = \\frac{\\lambda^2}{\\mu(\\mu - \\lambda)}$$

**Average Wait Time** (excluding service):
$$W_q = \\frac{\\lambda}{\\mu(\\mu - \\lambda)}$$

These formulas let us **validate** our simulation!

In [ ]:
# Example: Calculate theoretical predictions for λ=2, μ=3


def calculate_mm1_metrics(arrival_rate, service_rate):
    """Calculate theoretical metrics for M/M/1 queue."""
    lambda_rate = arrival_rate
    mu_rate = service_rate

    # Check stability
    rho = lambda_rate / mu_rate
    if rho >= 1:
        return "System is unstable! (rho >= 1)"

    # Calculate metrics
    L = rho / (1 - rho)  # Avg customers in system
    W = 1 / (mu_rate - lambda_rate)  # Avg time in system
    Lq = (rho**2) / (1 - rho)  # Avg queue length
    Wq = lambda_rate / (mu_rate * (mu_rate - lambda_rate))  # Avg wait time

    return {
        "Utilization (rho)": rho,
        "Avg in System (L)": L,
        "Avg Time in System (W)": W,
        "Avg in Queue (Lq)": Lq,
        "Avg Wait Time (Wq)": Wq,
    }


# Example calculation
metrics = calculate_mm1_metrics(arrival_rate=2.0, service_rate=3.0)
for key, value in metrics.items():
    print(f"{key}: {value:.4f}")

### The Exponential Distribution

**Why exponential?** It has the **memoryless property**:

> The probability of an event occurring in the next Δt time units is **independent** of how long you've already waited.

**Probability Density Function**:
$$f(t) = \\lambda e^{-\\lambda t}$$

**Mean**: $\\frac{1}{\\lambda}$

**In Python**: `np.random.exponential(1/rate)` or `random.expovariate(rate)`

In [ ]:
# Visualize exponential distribution for inter-arrival times
arrival_rate = 2.0  # 2 customers per hour
samples = np.random.exponential(1 / arrival_rate, 1000)

plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
plt.hist(samples, bins=50, density=True, alpha=0.7, label="Simulated")
x = np.linspace(0, 5, 100)
plt.plot(
    x, arrival_rate * np.exp(-arrival_rate * x), "r-", linewidth=2, label="Theoretical"
)
plt.xlabel("Inter-arrival Time (hours)")
plt.ylabel("Probability Density")
plt.title("Exponential Distribution (λ=2)")
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.hist(samples, bins=50, cumulative=True, density=True, alpha=0.7)
plt.xlabel("Inter-arrival Time (hours)")
plt.ylabel("Cumulative Probability")
plt.title("Cumulative Distribution")
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"Theoretical mean: {1 / arrival_rate:.4f}")
print(f"Simulated mean: {np.mean(samples):.4f}")

## Core DEVS Concepts: The DEVSimulator

The `DEVSimulator` is Mesa's pure discrete event simulator.

**Key Features**:
- **Float time units**: Continuous time (0.0, 1.3, 2.7, ...)
- **Event-driven**: Only executes scheduled events
- **Priority queue**: Events sorted by time, then priority
- **Weak references**: Automatic cleanup of canceled events

**Architecture**:
```
DEVSimulator
  ├── EventList (priority queue)
  │     └── SimulationEvent(time, function, priority)
  ├── setup(model) - Initialize with model
  ├── run_until(end_time) - Run until time
  └── run_for(time_delta) - Run for duration
```

### Event Scheduling Methods

The `DEVSimulator` provides three ways to schedule events:

1. **`schedule_event_now(function, priority=DEFAULT)`**
   - Schedule event for current time instant
   - Executes immediately when simulator processes current time

2. **`schedule_event_absolute(function, time, priority=DEFAULT)`**
   - Schedule event at specific simulation time
   - Time must be >= current simulation time

3. **`schedule_event_relative(function, time_delta, priority=DEFAULT)`**
   - Schedule event relative to current time
   - time_delta must be >= 0

**Arguments**:
- `function`: Callable to execute (method reference, not lambda!)
- `priority`: Event priority (HIGH=1, DEFAULT=5, LOW=10)
- `function_args`: List of arguments for function
- `function_kwargs`: Dictionary of keyword arguments

In [ ]:
# Simple example: Schedule three events
from mesa.experimental.devs import DEVSimulator


class SimpleModel(mesa.Model):
    def __init__(self, simulator):
        super().__init__()
        self.simulator = simulator
        self.events_executed = []

    def event_at_time_5(self):
        self.events_executed.append(("Event A", self.time))
        print(f"Event A executed at time {self.time}")

    def event_at_time_10(self):
        self.events_executed.append(("Event B", self.time))
        print(f"Event B executed at time {self.time}")

    def event_relative(self):
        self.events_executed.append(("Event C", self.time))
        print(f"Event C executed at time {self.time}")


# Create simulator and model
simulator = DEVSimulator()
model = SimpleModel(simulator)
simulator.setup(model)

# Schedule events
simulator.schedule_event_absolute(model.event_at_time_5, 5.0)
simulator.schedule_event_absolute(model.event_at_time_10, 10.0)
simulator.schedule_event_relative(model.event_relative, 7.5)  # at time 7.5

# Run simulation
simulator.run_until(15.0)

print(f"\nFinal time: {model.time}")
print(f"Events executed: {model.events_executed}")

### Event Priority

When multiple events are scheduled for the **same time**, priority determines execution order:

**Priority Levels**:
- `Priority.HIGH = 1` - Execute first
- `Priority.DEFAULT = 5` - Normal priority
- `Priority.LOW = 10` - Execute last

**Use Cases**:
- HIGH: Critical events (arrivals, failures)
- DEFAULT: Normal processing
- LOW: Cleanup, logging, statistics collection

In [ ]:
# Demonstrate priority ordering
class PriorityDemo(mesa.Model):
    def __init__(self, simulator):
        super().__init__()
        self.simulator = simulator

    def high_priority(self):
        print(f"  HIGH priority event at time {self.time}")

    def default_priority(self):
        print(f"  DEFAULT priority event at time {self.time}")

    def low_priority(self):
        print(f"  LOW priority event at time {self.time}")


simulator = DEVSimulator()
model = PriorityDemo(simulator)
simulator.setup(model)

# Schedule all at time 10.0 with different priorities
print("Scheduling events at time 10.0:")
simulator.schedule_event_absolute(model.default_priority, 10.0, Priority.DEFAULT)
simulator.schedule_event_absolute(model.low_priority, 10.0, Priority.LOW)
simulator.schedule_event_absolute(model.high_priority, 10.0, Priority.HIGH)

print("\nExecution order:")
simulator.run_until(11.0)

## Example: M/M/1 Bank Queue System

Let's build a complete queuing system simulation!

**Scenario**: A bank with one teller serving customers.

**System Dynamics**:
1. Customers arrive randomly (Poisson process, rate λ)
2. Inter-arrival times are exponential with mean 1/λ
3. Service times are exponential with mean 1/μ
4. One server, FIFO (first-in-first-out) queue
5. No customer abandons the queue

**Metrics to Track**:
- Wait time (time from arrival to service start)
- Time in system (wait time + service time)
- Queue length over time
- Server utilization

In [ ]:
class Customer(mesa.Agent):
    """A customer in the bank queue."""

    def __init__(self, model, arrival_time):
        super().__init__(model)
        self.arrival_time = arrival_time
        self.service_start_time = None
        self.service_end_time = None

    @property
    def wait_time(self):
        """Time spent waiting in queue."""
        if self.service_start_time is None:
            return None
        return self.service_start_time - self.arrival_time

    @property
    def service_time(self):
        """Time spent being served."""
        if self.service_end_time is None or self.service_start_time is None:
            return None
        return self.service_end_time - self.service_start_time

    @property
    def total_time(self):
        """Total time in system."""
        if self.service_end_time is None:
            return None
        return self.service_end_time - self.arrival_time

In [ ]:
class BankQueue(mesa.Model):
    """M/M/1 Queue: Single server with exponential arrivals and service."""

    def __init__(self, simulator, arrival_rate=2.0, service_rate=3.0):
        super().__init__()
        self.simulator = simulator
        self.arrival_rate = arrival_rate
        self.service_rate = service_rate

        # Queue and server state
        self.queue = []
        self.server_busy = False
        self.current_customer = None

        # Statistics
        self.completed_customers = []
        self.customer_count = 0

        # Schedule first arrival

    def schedule_next_arrival(self):
        """Schedule the next customer arrival."""
        # Exponential inter-arrival time
        inter_arrival_time = self.random.expovariate(self.arrival_rate)

        # Schedule arrival event
        self.simulator.schedule_event_relative(
            self.customer_arrival,
            inter_arrival_time,
            priority=Priority.HIGH,  # Arrivals are high priority
        )

    def customer_arrival(self):
        """Handle a customer arrival."""
        # Create new customer
        self.customer_count += 1
        customer = Customer(self, arrival_time=self.time)

        # Add to queue
        self.queue.append(customer)
        print(
            f"Time {self.time:.2f}: Customer {customer.unique_id} arrived. Queue length: {len(self.queue)}"
        )

        # If server is idle, start service immediately
        if not self.server_busy:
            self.start_service()

        # Schedule next arrival

    def start_service(self):
        """Start serving the next customer in queue."""
        if not self.queue:
            return  # No customers to serve

        # Take first customer from queue
        customer = self.queue.pop(0)
        customer.service_start_time = self.time
        self.current_customer = customer
        self.server_busy = True

        # Generate service time
        service_time = self.random.expovariate(self.service_rate)

        # Schedule service completion
        self.simulator.schedule_event_relative(
            self.finish_service,
            service_time,
            priority=Priority.DEFAULT,
            function_args=[customer],
        )

        print(
            f"Time {self.time:.2f}: Customer {customer.unique_id} service started (will take {service_time:.2f})"
        )

    def finish_service(self, customer):
        """Handle service completion."""
        customer.service_end_time = self.time
        self.completed_customers.append(customer)
        self.server_busy = False
        self.current_customer = None

        print(
            f"Time {self.time:.2f}: Customer {customer.unique_id} service completed. Wait time: {customer.wait_time:.2f}"
        )

        # Start serving next customer if any
        self.start_service()

### Running the Simulation

Let's simulate the bank queue for 100 time units with:
- λ = 2 customers/hour
- μ = 3 customers/hour
- ρ = 2/3 = 0.67 (stable system)

In [ ]:
# Create and run simulation
simulator = DEVSimulator()
model = BankQueue(simulator, arrival_rate=2.0, service_rate=3.0)
simulator.setup(model)

print("=" * 60)
print("Starting Bank Queue Simulation")
print("Arrival rate (λ): 2.0 customers/hour")
print("Service rate (μ): 3.0 customers/hour")
print("=" * 60)
print()

# Run for 100 time units
simulator.run_until(100.0)

print()
print("=" * 60)
print("Simulation Complete")
print(f"Total customers arrived: {model.customer_count}")
print(f"Total customers served: {len(model.completed_customers)}")
print(f"Customers still in queue: {len(model.queue)}")
print("=" * 60)

### Warm-Up Period

Notice the first few customers might experience different wait times because the system starts **empty**. This is called the **transient period**.

For accurate steady-state statistics, we should:
1. Run a warm-up period (discard early data)
2. Run multiple replications
3. Use longer simulation times

We'll analyze this in the next section!

In [ ]:
# Quick look at results
if model.completed_customers:
    wait_times = [c.wait_time for c in model.completed_customers]
    service_times = [c.service_time for c in model.completed_customers]
    total_times = [c.total_time for c in model.completed_customers]

    print(f"Average wait time: {np.mean(wait_times):.4f}")
    print(f"Average service time: {np.mean(service_times):.4f}")
    print(f"Average total time: {np.mean(total_times):.4f}")

## Statistical Analysis: Comparing Simulation vs Theory

To properly analyze the system, we'll:
1. Run a **warm-up period** (50 time units)
2. Collect data from the **steady-state period**
3. Run **multiple replications** for confidence
4. Compare simulation results with **theoretical predictions**

In [ ]:
def run_bank_simulation(arrival_rate, service_rate, run_time=200, warmup=50, seed=None):
    """Run one replication of the bank queue simulation."""
    simulator = DEVSimulator()
    model = BankQueue(simulator, arrival_rate, service_rate)

    if seed is not None:
        model.random.seed(seed)

    simulator.setup(model)
    simulator.run_until(run_time)

    # Filter customers completed after warm-up
    steady_state_customers = [
        c for c in model.completed_customers if c.arrival_time >= warmup
    ]

    if not steady_state_customers:
        return None

    return {
        "wait_times": [c.wait_time for c in steady_state_customers],
        "total_times": [c.total_time for c in steady_state_customers],
        "n_customers": len(steady_state_customers),
    }


# Run 20 replications
n_reps = 20
arrival_rate = 2.0
service_rate = 3.0

print(f"Running {n_reps} replications...")
results = []
for i in range(n_reps):
    result = run_bank_simulation(
        arrival_rate, service_rate, run_time=200, warmup=50, seed=i
    )
    if result:
        results.append(result)
    if (i + 1) % 5 == 0:
        print(f"  Completed {i + 1}/{n_reps}")

print(f"Successfully completed {len(results)} replications")

In [ ]:
# Aggregate all wait times across replications
all_wait_times = []
all_total_times = []
mean_wait_times = []

for result in results:
    all_wait_times.extend(result["wait_times"])
    all_total_times.extend(result["total_times"])
    mean_wait_times.append(np.mean(result["wait_times"]))

# Calculate statistics
sim_mean_wait = np.mean(all_wait_times)
sim_std_wait = np.std(all_wait_times)
sim_mean_total = np.mean(all_total_times)

print("Simulation Results (Steady-State):")
print(f"  Mean wait time: {sim_mean_wait:.4f}")
print(f"  Std wait time: {sim_std_wait:.4f}")
print(f"  Mean total time: {sim_mean_total:.4f}")
print(f"  Total customers analyzed: {len(all_wait_times)}")

In [ ]:
# Calculate theoretical values
lambda_val = arrival_rate
mu_val = service_rate
rho = lambda_val / mu_val

# Theoretical metrics
theory_Wq = lambda_val / (mu_val * (mu_val - lambda_val))
theory_W = 1 / (mu_val - lambda_val)
theory_Lq = (rho**2) / (1 - rho)
theory_L = rho / (1 - rho)

print("\nTheoretical Predictions (M/M/1):")
print(f"  Mean wait time (Wq): {theory_Wq:.4f}")
print(f"  Mean total time (W): {theory_W:.4f}")
print(f"  Mean queue length (Lq): {theory_Lq:.4f}")
print(f"  Mean in system (L): {theory_L:.4f}")

print("\nComparison:")
print(
    f"  Wait time - Simulation: {sim_mean_wait:.4f}, Theory: {theory_Wq:.4f}, Error: {abs(sim_mean_wait - theory_Wq) / theory_Wq * 100:.2f}%"
)
print(
    f"  Total time - Simulation: {sim_mean_total:.4f}, Theory: {theory_W:.4f}, Error: {abs(sim_mean_total - theory_W) / theory_W * 100:.2f}%"
)

In [ ]:
# Visualize wait time distribution
plt.figure(figsize=(14, 5))

# Histogram of wait times
plt.subplot(1, 3, 1)
plt.hist(all_wait_times, bins=50, density=True, alpha=0.7, label="Simulated")
plt.axvline(
    sim_mean_wait,
    color="blue",
    linestyle="--",
    linewidth=2,
    label=f"Sim Mean: {sim_mean_wait:.2f}",
)
plt.axvline(
    theory_Wq,
    color="red",
    linestyle="--",
    linewidth=2,
    label=f"Theory Mean: {theory_Wq:.2f}",
)
plt.xlabel("Wait Time")
plt.ylabel("Probability Density")
plt.title("Wait Time Distribution")
plt.legend()
plt.grid(True, alpha=0.3)

# Box plot of mean wait times per replication
plt.subplot(1, 3, 2)
plt.boxplot(mean_wait_times)
plt.axhline(theory_Wq, color="red", linestyle="--", linewidth=2, label="Theory")
plt.ylabel("Mean Wait Time")
plt.title(f"Distribution Across {len(results)} Replications")
plt.grid(True, alpha=0.3)
plt.legend()

# Q-Q plot
plt.subplot(1, 3, 3)
from scipy import stats

stats.probplot(all_wait_times, dist="expon", plot=plt)
plt.title("Q-Q Plot (Exponential)")
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### Analysis Conclusion

Our simulation results match the theoretical predictions very closely!

**Key Takeaways**:
1. Simulation validates queuing theory formulas
2. Warm-up period removes transient effects
3. Multiple replications reduce variance
4. DEVS efficiently simulates sparse events

The small differences come from:
- Finite simulation time
- Random sampling variation
- Warm-up period approximation

## Advanced Topics

### Event Cancellation

Sometimes you need to cancel a scheduled event (e.g., customer leaves queue).

In [ ]:
class CancellationDemo(mesa.Model):
    def __init__(self, simulator):
        super().__init__()
        self.simulator = simulator
        self.cancelled = False

    def event_to_cancel(self):
        print(f"This event was NOT cancelled (time={self.time})")

    def cancellation_decision(self):
        # Cancel the event scheduled for time 10
        if hasattr(self, "scheduled_event"):
            self.simulator.cancel_event(self.scheduled_event)
            print(f"Event cancelled at time {self.time}")
            self.cancelled = True


simulator = DEVSimulator()
model = CancellationDemo(simulator)
simulator.setup(model)

# Schedule an event at time 10
model.scheduled_event = simulator.schedule_event_absolute(model.event_to_cancel, 10.0)

# Schedule cancellation at time 5
simulator.schedule_event_absolute(model.cancellation_decision, 5.0)

# Run
simulator.run_until(15.0)

if model.cancelled:
    print("Event was successfully cancelled")
else:
    print("Event was NOT cancelled")

### Warning: Weak References

**DO NOT use lambda functions** with event scheduling!

```python
# WRONG - This will fail silently!
simulator.schedule_event_relative(
    lambda: print("Hello"),
    5.0
)

# CORRECT - Use method references
def my_event(self):
    print("Hello")

simulator.schedule_event_relative(self.my_event, 5.0)
```

**Why?** Events use weak references. Lambdas are garbage collected immediately!

### Debugging DEVS Models

**Common Issues**:

1. **Events don't execute**: Check if you used lambda (use method reference instead)
2. **Time doesn't advance**: Make sure you scheduled at least one event
3. **Queue order wrong**: Check priority levels (HIGH < DEFAULT < LOW)
4. **Infinite loop**: Verify each event schedules correctly

**Debugging Tools**:
```python
# Check event list contents
print(f"Events pending: {len(simulator.event_list)}")
print(f"Next events: {simulator.event_list.peak_ahead(3)}")

# Print event details
for event in simulator.event_list.peak_ahead(5):
    print(f"  Time: {event.time}, Priority: {event.priority}")
```

## Exercises

### Exercise 1: Multiple Servers (M/M/c Queue)

Modify the `BankQueue` model to have **c servers** instead of one.

**Changes needed**:
- Track which servers are busy (list or counter)
- Modify `start_service()` to check for available servers
- Update theoretical formulas for M/M/c

**Hint**: With c servers, the system is stable if ρ = λ/(c·μ) < 1

In [ ]:
# FIXME: Implement M/M/c queue with c=3 servers
class MultiServerBank(mesa.Model):
    def __init__(self, simulator, arrival_rate=5.0, service_rate=2.0, num_servers=3):
        super().__init__()
        self.simulator = simulator
        self.arrival_rate = arrival_rate
        self.service_rate = service_rate
        self.num_servers = num_servers

        # FIXME: Track server states
        self.servers_busy = 0  # Count of busy servers
        self.queue = []

        # TODO: Implement the rest...

### Exercise 2: Priority Customers

Add a **VIP lane** where priority customers:
- Get served before regular customers
- Have different arrival rate

**Hints**:
- Use two queues: `vip_queue` and `regular_queue`
- In `start_service()`, check VIP queue first
- Schedule two arrival processes

### Exercise 3: Customer Balking

Customers **leave if queue is too long** (balking).

**Changes**:
- In `customer_arrival()`, check queue length
- If queue > threshold, customer leaves (track this!)
- Calculate **balking probability** from simulation

**Question**: How does balking affect wait times?

## Next Steps

Congratulations! You've mastered pure discrete event simulation with `DEVSimulator`.

**You learned**:
- When to use DEVS vs tick-based ABM
- Mathematical foundations (queuing theory, exponential distributions)
- Building event-driven models
- Validating simulations with theoretical predictions
- Statistical analysis of simulation results

### Continue Learning

**Tutorial 12: Hybrid ABM-DEVS Models**
- Learn `ABMSimulator` for combining ticks and events
- Build spatial models with event scheduling
- Optimize performance with hybrid approaches

**Additional Resources**:
- [Mesa DEVS API Documentation](https://mesa.readthedocs.io/latest/apis/experimental.html#devs)
- [mesa-examples DEVS Models](https://github.com/projectmesa/mesa-examples)
- [Queuing Theory Reference](https://en.wikipedia.org/wiki/Queueing_theory)

## Happy Modeling!

This tutorial is a work in progress. If you found any errors, have suggestions, or need clarification, please:
- Open an issue: [Mesa GitHub Issues](https://github.com/projectmesa/mesa/issues)
- Ask in chat: [Mesa Matrix Chat](https://matrix.to/#/#project-mesa:matrix.org)
- Check documentation: [Mesa Docs](https://mesa.readthedocs.io/)

**References**

[QueueingTheory] Gross, D., et al. "Fundamentals of Queueing Theory." 4th Edition, Wiley, 2008.  
[DEVS] Zeigler, B. P. "Theory of Modeling and Simulation." 3rd Edition, Academic Press, 2018.